###  Write testutility.py file

In [1]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


### Write YAML file

In [2]:
%%writefile file.yaml
file_type: csv
dataset_name: dataset
file_name: custom_1988_2020
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - yr
    - exp_imp
    - hs_code
    - customs
    - country
    - q1
    - q2
    - value

Overwriting file.yaml


In [3]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [4]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'dataset',
 'file_name': 'custom_1988_2020',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['yr',
  'exp_imp',
  'hs_code',
  'customs',
  'country',
  'q1',
  'q2',
  'value']}

## 1st method of reading the csv file, through 'pandas' library

In [5]:
# read the file using config file
import pandas as pd
file_type = config_data['file_type']
source_file = "D:/" + config_data['file_name'] + f'.{file_type}'

%time dataset = pd.read_csv(source_file,config_data['inbound_delimiter'])
dataset.head()

C:\Users\Enias\anaconda3\lib\site-packages\IPython\core\magics\execution.py:1321: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code, glob, local_ns)


Wall time: 1min 14s


,yr,exp_imp,hs_code,customs,country,Q1,Q2,value
0,198801,1,103,100,190,0,35843,34353
1,198801,1,103,100,120991000,0,1590,4154
2,198801,1,103,100,210390900,0,4500,2565
3,198801,1,103,100,220890200,0,3000,757
4,198801,1,103,100,240220000,0,26000,40668


In [6]:
#validate the header of the file
util.col_header_val(dataset,config_data)

column name and column length validation passed


1

In [7]:
print("columns of csv file are:" ,dataset.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of csv file are: Index(['yr', 'exp_imp', 'hs_code', 'customs', 'country', 'q1', 'q2', 'value'], dtype='object')
columns of YAML are: ['yr', 'exp_imp', 'hs_code', 'customs', 'country', 'q1', 'q2', 'value']


In [8]:
if util.col_header_val(dataset,config_data)==0:
    print("validation failed")
    os.remove("D:\custom_1988_2020.csv")
else:
    print("col validation passed")
    

column name and column length validation passed
col validation passed


### Further actions after passing column validation

In [9]:
%time dataset.groupby("exp_imp").exp_imp.count()

Wall time: 2.23 s


exp_imp
1    69088368
2    44518954
Name: exp_imp, dtype: int64

In [10]:
%time dataset.yr.max()

Wall time: 111 ms


202012

In [11]:
%time dataset.value.mean()

Wall time: 111 ms


32408.302375721876

In [12]:
%time dataset.value.std()

Wall time: 214 ms


376908.21176299546

In [18]:
%time dataset.describe()

Wall time: 29.7 s


,yr,exp_imp,hs_code,customs,country,q1,q2,value
count,1.136073e+08,1.136073e+08,1.136073e+08,1.136073e+08,1.136073e+08,1.136073e+08,1.136073e+08,1.136073e+08
mean,2.005126e+05,1.391867e+00,1.932035e+02,3.133187e+02,6.545763e+08,4.707675e+04,1.271670e+05,3.240830e+04
std,9.286563e+02,4.881672e-01,1.210249e+02,1.797740e+02,2.521764e+08,2.914592e+07,4.571867e+06,3.769082e+05
min,1.988010e+05,1.000000e+00,1.030000e+02,1.000000e+02,1.100000e+01,0.000000e+00,0.000000e+00,3.000000e+01
25%,1.997110e+05,1.000000e+00,1.060000e+02,1.040000e+02,4.016930e+08,0.000000e+00,9.800000e+01,7.200000e+02
50%,2.005110e+05,1.000000e+00,1.230000e+02,3.000000e+02,7.415210e+08,0.000000e+00,7.150000e+02,2.475000e+03
75%,2.013090e+05,2.000000e+00,2.220000e+02,4.230000e+02,8.517900e+08,7.000000e+00,6.720000e+03,1.005200e+04
max,2.020120e+05,2.000000e+00,7.030000e+02,9.080000e+02,9.706000e+08,1.255000e+11,1.885790e+09,1.832784e+08


In [14]:
x = dataset.q2.values
y = dataset.value.values

In [15]:
x = x.reshape(len(x),1)
y = y.reshape(len(y),1)

In [16]:
from sklearn.linear_model import LinearRegression

In [17]:
%time model = LinearRegression().fit(x,y)

Wall time: 5.23 s


In [19]:
model.score(x,y)

0.1824214630085732

In [20]:
print('intercept:', model.intercept_)

intercept: [27930.60932872]


In [21]:
print('slope:', model.coef_)

slope: [[0.03521113]]


## 2nd method of reading the csv file, through 'dask' library
### We restart the kernel and run the first, second and third cells of the notebook again

### Then we load the 'dask' library

In [4]:
import dask.dataframe as dd
# read the file using config file
file_type = config_data['file_type']
source_file = "D:/" + config_data['file_name'] + f'.{file_type}'

%time dataset = dd.read_csv(source_file)
dataset.head()

Wall time: 15.6 ms


,yr,exp_imp,hs_code,customs,country,Q1,Q2,value
0,198801,1,103,100,190,0,35843,34353
1,198801,1,103,100,120991000,0,1590,4154
2,198801,1,103,100,210390900,0,4500,2565
3,198801,1,103,100,220890200,0,3000,757
4,198801,1,103,100,240220000,0,26000,40668


In [5]:
print("columns of csv file are:" ,dataset.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of csv file are: Index(['yr', 'exp_imp', 'hs_code', 'customs', 'country', 'Q1', 'Q2', 'value'], dtype='object')
columns of YAML are: ['yr', 'exp_imp', 'hs_code', 'customs', 'country', 'q1', 'q2', 'value']


### Further actions after passing column validation

In [6]:
%time dataset.groupby("exp_imp").exp_imp.count().compute()

Wall time: 22.9 s


exp_imp
1    69088368
2    44518954
Name: exp_imp, dtype: int64

In [7]:
%time dataset.yr.max().compute()

Wall time: 19.4 s


202012

In [8]:
%time dataset.value.mean().compute()

Wall time: 19.7 s


32408.302375721876

In [9]:
%time dataset.describe().compute()

Wall time: 35.1 s


,yr,exp_imp,hs_code,customs,country,Q1,Q2,value
count,1.136073e+08,1.136073e+08,1.136073e+08,1.136073e+08,1.136073e+08,1.136073e+08,1.136073e+08,1.136073e+08
mean,2.005126e+05,1.391867e+00,1.932035e+02,3.133187e+02,6.545763e+08,4.707675e+04,1.271670e+05,3.240830e+04
std,9.286563e+02,4.881672e-01,1.210249e+02,1.797740e+02,2.521764e+08,2.914592e+07,4.571867e+06,3.769082e+05
min,1.988010e+05,1.000000e+00,1.030000e+02,1.000000e+02,1.100000e+01,0.000000e+00,0.000000e+00,3.000000e+01
25%,1.997100e+05,1.000000e+00,1.110000e+02,1.040000e+02,4.820109e+08,0.000000e+00,1.160000e+02,7.690000e+02
50%,2.005110e+05,1.000000e+00,2.050000e+02,3.030000e+02,8.307100e+08,0.000000e+00,9.210000e+02,2.804000e+03
75%,2.013080e+05,2.000000e+00,3.020000e+02,5.000000e+02,8.526919e+08,2.600000e+01,9.000000e+03,1.208000e+04
max,2.020120e+05,2.000000e+00,7.030000e+02,9.080000e+02,9.706000e+08,1.255000e+11,1.885790e+09,1.832784e+08


In [6]:
from dask_ml.linear_model import LinearRegression

In [13]:
from dask_glm.datasets import make_regression

In [ ]:
model = LinearRegression()

In [8]:
x = dataset.Q2.values
y = dataset.value.values

In [16]:
x,y = make_regression()

In [21]:
%time model = LinearRegression().fit(x,y)

Wall time: 22.5 s


## 3rd method of reading the csv file, through 'modin' library
### We restart the kernel and run the first, second and third cells of the notebook again
### Then we load the 'modin' library

In [4]:
from dask.distributed import Client

# set up cluster and workers
client = Client(n_workers=4, 
                threads_per_worker=1,
                memory_limit='12GB')

import modin.pandas as mpd
# read the file using config file
file_type = config_data['file_type']
source_file = "D:/" + config_data['file_name'] + f'.{file_type}'

%time dataset = mpd.read_csv(source_file,config_data['inbound_delimiter'])
dataset.head()

Wall time: 52.7 s


,yr,exp_imp,hs_code,customs,country,Q1,Q2,value
0,198801,1,103,100,190,0,35843,34353
1,198801,1,103,100,120991000,0,1590,4154
2,198801,1,103,100,210390900,0,4500,2565
3,198801,1,103,100,220890200,0,3000,757
4,198801,1,103,100,240220000,0,26000,40668


### Further actions after passing column validation

In [5]:
%time dataset.groupby("exp_imp").exp_imp.count()

Wall time: 8.31 s


exp_imp
1    69088368
2    44518954
Name: exp_imp, dtype: int64

In [6]:
%time dataset.yr.max()

Wall time: 1.63 s


202012

In [7]:
%time dataset.value.mean()

Wall time: 3.03 s


32408.302375721876

In [8]:
%time dataset.value.std()

Wall time: 3.05 s


376908.21176299546

In [9]:
x = dataset["Q2"].values
y = dataset["value"].values

In [10]:
x = x.reshape(len(x),1)
y = y.reshape(len(y),1)

In [11]:
from sklearn.linear_model import LinearRegression

%time model = LinearRegression().fit(x,y)


Wall time: 4.17 s


In [12]:
print('intercept:', model.intercept_)

intercept: [27930.60932872]


In [13]:
print('slope:', model.coef_)

slope: [[0.03521113]]


In [ ]:
# dataset.describe()

## 4th method of reading the csv file, through 'vaex' library
### We restart the kernel and run the first, second and third cells of the notebook again
### Then we load the 'vaex' library

In [4]:
import vaex


In [5]:
file_type = config_data['file_type']
source_file = "D:/" + config_data['file_name'] + f'.{file_type}'

source_file

'D:/custom_1988_2020.csv'

In [6]:
%time dataset = vaex.from_csv(source_file,config_data['inbound_delimiter'])

Wall time: 1min 27s


In [7]:
%time dataset.groupby('exp_imp' , agg=[vaex.agg.count(dataset.exp_imp)])

Wall time: 387 ms


#,exp_imp,exp_imp_count
0,1,69088368
1,2,44518954


In [8]:
%time dataset.yr.max()

Wall time: 111 ms


array(202012, dtype=int64)

In [9]:
%time dataset.value.mean()

Wall time: 245 ms


array(32408.30237572)

In [10]:
%time dataset.value.std()

Wall time: 660 ms


376908.21010742965

In [11]:
%time dataset.describe()

Wall time: 7.47 s


,yr,exp_imp,hs_code,customs,country,Q1,Q2,value,index
data_type,int64,int64,int64,int64,int64,int64,int64,int64,int64
count,113607322,113607322,113607322,113607322,113607322,113607322,113607322,113607322,113607322
NA,0,0,0,0,0,0,0,0,0
mean,200512.6231617008,1.3918669432239588,193.20352292962244,313.31866609794747,654576293.6521226,47076.754848697165,127166.98737165902,32408.302375721876,56803660.5
std,928.656401,0.488167,121.024864,179.77396,252176352.235584,29145919.720134,4571867.433452,376908.210107,32795608.969313
min,198801,1,103,100,11,0,0,30,0
max,202012,2,703,908,970600000,125500000000,1885789678,183278402,113607321


In [12]:
import vaex.ml
from vaex.ml.sklearn import Predictor
from sklearn.linear_model import LinearRegression

In [13]:
features = ['Q2']
model = Predictor(model=LinearRegression(), features=features, target='value', prediction_name='pred')

In [14]:
%time model = model.fit(dataset)

Wall time: 5.16 s


### It is interesting to note how faster 'vaex' library would be if the file was in '.hdf5' format, as we can see below

In [15]:
%time dataset =  vaex.open('D:\custom_1988_2020.csv.hdf5')

Wall time: 1.66 s


## Write the file in pipe delimited format

In [ ]:
dataset.to_csv(r"D:\custom_1988_2020.txt",sep="|",index=False)

## Save the file in gz format

In [ ]:
import gzip
with open(r"D:\custom_1988_2020.txt", 'rb') as f_in, gzip.open(r"D:\custom_1988_2020.txt.gz", 'wb') as f_out:
    f_out.writelines(f_in)